In [26]:
import pandas as pd
import random, datetime
branches_list = ['0010','0020','0030','0040','0050','0060','0070']
is_pep_list = ['hni','hne','pep','Yes','No', 'Adverse']
data = pd.DataFrame(columns= ['branch','ispep'])
for i in range(1000):
    branch = random.choice(branches_list)
    is_pep = random.choice(is_pep_list)
    data.loc[len(data)] = [branch,is_pep]
data

,branch,ispep
0,0030,hni
1,0050,hni
2,0040,Adverse
3,0060,pep
4,0050,hni
...,...,...
995,0050,hni
996,0020,No
997,0060,hne
998,0010,pep


In [27]:
data.pivot_table(index = 'branch', columns = 'ispep',aggfunc = len, margins = True, margins_name = 'subtotal').sort_values('subtotal')

ispep,Adverse,No,Yes,hne,hni,pep,subtotal
branch,,,,,,,
0060,29,22,20,13,22,25,131
0070,16,22,24,31,19,21,133
0050,23,23,17,29,25,19,136
0010,25,13,26,23,19,33,139
0040,19,23,26,22,33,25,148
0020,32,26,28,22,26,16,150
0030,28,34,19,24,30,28,163
subtotal,172,163,160,164,174,167,1000


In [28]:
branches_list = ['0010','0020','0030','0040','0050','0060','0070']
risk_category = ['a','b','c']
start_date, end_date = datetime.date(2015,1,1), datetime.date(2025,1,1)
data = pd.DataFrame(columns= ['branch','risk','review_date'])
for i in range(1000):
    branch = random.choice(branches_list)
    risk = random.choice(risk_category)
    review_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
    data.loc[len(data)] = [branch, risk, review_date]
data

,branch,risk,review_date
0,0070,a,2015-08-14
1,0070,b,2022-02-26
2,0060,c,2019-11-04
3,0070,b,2020-01-15
4,0030,b,2017-06-23
...,...,...,...
995,0030,a,2016-02-09
996,0040,a,2015-01-30
997,0030,c,2015-10-06
998,0030,a,2020-09-16


In [32]:
from dateutil.relativedelta import relativedelta
data['years_to_add'] = data['risk'].map({'a': 8, 'b': 5, 'c': 1})
data['to_review_date'] = data['review_date'] + data['years_to_add'].apply(lambda x : relativedelta( years = int(x)))
data.loc[:,'to_review_date'] = pd.to_datetime(data['to_review_date'])
def checkcdd(dates):
    for date in dates:
        ref_date = pd.to_datetime(date)
        fil = data [data['to_review_date'] < ref_date]
    return fil

In [70]:
dates = ['2025-07-17','2025-01-01','2024-06-01','2024-01-01']
summary = pd.DataFrame()
main_piv = pd.DataFrame()
for date in dates:
    ref_date = pd.to_datetime(date)
    fil = data [data['to_review_date'] < ref_date]
    piv = fil.pivot_table(index = 'branch',columns = 'risk', values = 'review_date',aggfunc = len,margins = True)
    piv.columns = [ x + date for x in piv.columns]
    main_piv[piv.columns] = piv
    counts = fil.risk.value_counts()
    summary[date] = counts
summary
main_piv

,a2025-07-17,b2025-07-17,c2025-07-17,All2025-07-17,a2025-01-01,b2025-01-01,c2025-01-01,All2025-01-01,a2024-06-01,b2024-06-01,c2024-06-01,All2024-06-01,a2024-01-01,b2024-01-01,c2024-01-01,All2024-01-01
branch,,,,,,,,,,,,,,,,
0010,9,39,35,83,9,38,29,76,7,35,28,70,6,32,28,66
0020,9,21,44,74,7,18,44,69,4,17,41,62,4,16,39,59
0030,15,34,57,106,11,34,50,95,6,30,49,85,3,28,46,77
0040,13,30,47,90,8,29,44,81,6,27,40,73,6,24,39,69
0050,11,30,40,81,10,29,38,77,7,26,36,69,5,23,33,61
0060,18,25,43,86,15,22,40,77,12,16,37,65,7,14,36,57
0070,12,16,46,74,7,15,44,66,6,15,39,60,3,12,37,52
All,87,195,312,594,67,185,289,541,48,166,270,484,34,149,258,441


In [30]:
data1 = checkcdd('2025-07-17')
data1.pivot_table(index = 'branch',columns = 'risk', values = 'review_date',aggfunc = len,margins = True)

risk,a,b,c,All
branch,,,,
0010,9,39,35,83
0020,9,21,44,74
0030,15,34,57,106
0040,13,30,47,90
0050,11,30,40,81
0060,18,25,43,86
0070,12,16,46,74
All,87,195,312,594


In [31]:
data2 = checkcdd('2025-1-1')
data2.pivot_table(index = 'branch',columns = 'risk', values = 'review_date',aggfunc = len,margins = True)

risk,a,b,c,All
branch,,,,
0010,9,38,29,76
0020,7,18,44,69
0030,11,34,50,95
0040,8,29,44,81
0050,10,29,38,77
0060,15,22,40,77
0070,7,15,44,66
All,67,185,289,541


## Next steps 🏃

This is just a short introduction to JupyterLab and Jupyter Notebooks. See below for some more ways to interact with tools in the Jupyter ecosystem, and its community.

### Other notebooks in this demo

Here are some other notebooks in this demo. Each of the items below corresponds to a file or folder in the **file browser to the left**.

- [**`Lorenz.ipynb`**](Lorenz.ipynb) uses Python to demonstrate interactive visualizations and computations around the [Lorenz system](https://en.wikipedia.org/wiki/Lorenz_system). It shows off basic Python functionality, including more visualizations, data structures, and scientific computing libraries.
- [**`sqlite.ipynb`**](sqlite.ipynb) demonstrates how an in-browser sqlite kernel to run your own SQL commands from the notebook. It uses the [jupyterlite/xeus-sqlite-kernel](https://github.com/jupyterlite/xeus-sqlite-kernel).

### Other sources of information in Jupyter

- **More on using JupyterLab**: See [the JupyterLab documentation](https://jupyterlab.readthedocs.io/en/stable/) for more thorough information about how to install and use JupyterLab.
- **More interactive demos**: See [try.jupyter.org](https://try.jupyter.org) for more interactive demos with the Jupyter ecosystem.
- **Learn more about Jupyter**: See [the Jupyter community documentation](https://docs.jupyter.org) to learn more about the project, its community and tools, and how to get involved.
- **Join our discussions**: The [Jupyter Community Forum](https://discourse.jupyter.org) is a place where many in the Jupyter community ask questions, help one another, and discuss issues around interactive computing and our ecosystem.